10

In [32]:
from geopy.geocoders import Nominatim
import folium
import pandas as pd

location_list = [['Bucktown', 41.926897, -87.676661],
                 'Boystown, Chicago',
                 ['Roscoe Village', 41.943280, -87.679406],
                 'Logan Square, Chicago',
                 'Wicker Park, Chicago',
                 ['Lincoln Park', 41.925913, -87.648806],
                 'West Town, Chicago,',
                 'Uptown, Chicago',
                 'Near North Side, Chicago',
                 ['River West', 41.895065, -87.650236]]

for x, address in enumerate(location_list):
    if type(address) is not list:
        geolocator = Nominatim(user_agent="chi_explorer")
        location = geolocator.geocode(address)
        location_list[x] = [address.split(',')[0], location.latitude, location.longitude]
location_list

[['Bucktown', 41.926897, -87.676661],
 ['Boystown', 41.9438833, -87.6492669],
 ['Roscoe Village', 41.94328, -87.679406],
 ['Logan Square', 41.9285683, -87.7067931],
 ['Wicker Park', 41.9078018, -87.6781551],
 ['Lincoln Park', 41.925913, -87.648806],
 ['West Town', 41.9014213, -87.6861661],
 ['Uptown', 41.9666299, -87.6555458],
 ['Near North Side', 41.9000327, -87.6344975],
 ['River West', 41.895065, -87.650236]]

In [5]:
CLIENT_ID = '4N2OKUVXD1N1RQYQJOCBRGF1GBGOML5KLC4WV1GNX0JH3U3S' 
CLIENT_SECRET = 'YZSH4L5VMESUMFUXNAP3AIZ10HZPZMKLV05QRGKA40I3JCVX' 
VERSION = '20180605'

In [14]:
import requests
def getFoodVenues(location_list, radius=805,limit=50):  
    food_list=[]
    for name, l1, l2 in location_list: 
        # create the API request URL
        for offset in [0,50,100]:
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section=food&limit={}&offset={}'.format(
                CLIENT_ID,CLIENT_SECRET,VERSION, 
                l1,l2, 
                radius, 
                limit,offset)  
            # make the GET request
            results=requests.get(url)
            #print(results)
            results=results.json()["response"]['groups'][0]['items']
            #print(name, len(results))
            for item in results:
                food_list.append([name,l1,l2, 
                item['venue']['name'], 
                item['venue']['id'],
                item['venue']['location']['lat'], 
                item['venue']['location']['lng'],  
                item['venue']['categories'][0]['shortName']])
            if len(results) < 50:
                break

    venue_list=pd.DataFrame(food_list)
    venue_list.columns=['Location','Latitude','Longitude', 
                  'Business','id',
                  'Business Latitude','Business Longitude','Business Category']
    
    return venue_list

In [33]:
df_food_bus = getFoodVenues(location_list)
df_food_bus

,Location,Latitude,Longitude,Business,id,Business Latitude,Business Longitude,Business Category
0,Bucktown,41.926897,-87.676661,Vienna Beef Factory Store & Cafe,4b2fc693f964a520b0ef24e3,41.926136,-87.677651,Hot Dogs
1,Bucktown,41.926897,-87.676661,CRĪO,5355d7bb498eeed72bc2a0b7,41.927030,-87.672064,Latin American
2,Bucktown,41.926897,-87.676661,Taco Bell,4a7bbb8ff964a520b7eb1fe3,41.928577,-87.673296,Fast Food
3,Bucktown,41.926897,-87.676661,The Bristol,49ebe3cbf964a5203f671fe3,41.921223,-87.677758,New American
4,Bucktown,41.926897,-87.676661,Asian Cajun,56b2b238498ec9031ecf5c13,41.926061,-87.668434,Seafood
...,...,...,...,...,...,...,...,...
802,River West,41.895065,-87.650236,Monty's Tap,5d145d2eb55cb80023dc8570,41.896895,-87.642177,Restaurant
803,River West,41.895065,-87.650236,Honey Bear Ham Company,4ced6c930acea35d34c4e3ae,41.891322,-87.657020,American
804,River West,41.895065,-87.650236,Texican,58af62359435a903b95d578f,41.898403,-87.642824,Tex-Mex
805,River West,41.895065,-87.650236,Windy City Espresso,4ec94b8c9a52756c57d5c5a0,41.898777,-87.658196,Food Truck


In [36]:
len(df_food_bus.id.unique())

773

In [37]:
def pull_likes(ids):
    like_list = []
    for v_id in ids:
        url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(
                v_id,CLIENT_ID,CLIENT_SECRET,VERSION,)
        results=requests.get(url).json()["response"]['likes']['count']
        like_list.append([v_id, results])
        
    df_list=pd.DataFrame(like_list)
    df_list.columns=['id','Likes']
    return df_list


In [38]:
df_likes = pull_likes(df_food_bus.id.unique())
df_food_bus = pd.merge(df_food_bus, df_likes, on="id")
df_food_bus

,Location,Latitude,Longitude,Business,id,Business Latitude,Business Longitude,Business Category,Likes
0,Bucktown,41.926897,-87.676661,Vienna Beef Factory Store & Cafe,4b2fc693f964a520b0ef24e3,41.926136,-87.677651,Hot Dogs,89
1,Bucktown,41.926897,-87.676661,CRĪO,5355d7bb498eeed72bc2a0b7,41.927030,-87.672064,Latin American,34
2,Bucktown,41.926897,-87.676661,Taco Bell,4a7bbb8ff964a520b7eb1fe3,41.928577,-87.673296,Fast Food,111
3,Bucktown,41.926897,-87.676661,The Bristol,49ebe3cbf964a5203f671fe3,41.921223,-87.677758,New American,229
4,Bucktown,41.926897,-87.676661,Asian Cajun,56b2b238498ec9031ecf5c13,41.926061,-87.668434,Seafood,17
...,...,...,...,...,...,...,...,...,...
802,River West,41.895065,-87.650236,Recess,5e2fc6820af4890008de55a4,41.889435,-87.649473,Restaurant,0
803,River West,41.895065,-87.650236,Four Belly (Asian Street Food),539f9208498e792cf15c78c5,41.898770,-87.656285,Asian,1
804,River West,41.895065,-87.650236,Honey Bear Ham Company,4ced6c930acea35d34c4e3ae,41.891322,-87.657020,American,0
805,River West,41.895065,-87.650236,Windy City Espresso,4ec94b8c9a52756c57d5c5a0,41.898777,-87.658196,Food Truck,0


In [39]:
df_food_bus.to_csv('food_list.csv', index=False)

In [40]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
colors_array = cm.rainbow(np.linspace(0, 1, len(location_list)))
rainbow = {}
for x, each in enumerate(location_list):
    rainbow[each[0]] = colors.rgb2hex(colors_array[x])

In [41]:
map_t = folium.Map(location=[location_list[0][1], location_list[0][2]], zoom_start=12)

# add markers to map
for lat, lng, location in zip(df_food_bus['Business Latitude'], df_food_bus['Business Longitude'], df_food_bus['Location']):
    folium.CircleMarker([lat, lng],radius=5,color=rainbow[location],fill=True,fill_color=rainbow[location],fill_opacity=0.7,parse_html=False).add_to(map_t)  
    
map_t